In [1]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from config import db_password
from flask import Flask
from flask_sqlalchemy import sqlalchemy
import os
import re
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [5]:
path = 'Resources/'
funds = pd.read_csv(path +'funds_clean.csv', index_col=0)
objects = pd.read_csv(path + 'objects_clean.csv', index_col=0)
funds

C:\Users\phsstudent\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (3,5,7,8,9,10,11,13,14,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,object_id,name,funded_at,raised_amount,raised_currency_code
0,f:371,Second Fund,2008-12-16,300000000.0,USD
1,f:17,Sequoia Israel Fourth Fund,2008-12-17,200750000.0,USD
2,f:951,Tenth fund,2008-08-11,650000000.0,USD
3,f:192,New funds acquire,NaN,625000000.0,USD
4,f:519,Third fund,2008-05-20,200000000.0,USD
...,...,...,...,...,...
1559,f:101,Fund V,2013-12-11,150000000.0,USD
1560,f:7331,JANVEST Technologies LP I,2013-03-19,5.0,USD
1561,f:7791,Fund II,2013-12-11,5000000.0,USD
1562,f:15091,Fund I,2013-12-11,1100000.0,USD


In [6]:
fin_orgs = objects[objects['object_id'].str.contains('f:', regex=True)]
fin_orgs

,object_id,entity_type,parent_id,name,category_code,status,founded_at,closed_at,country_code,state_code,city,region,first_investment_at,last_investment_at,investment_rounds,invested_companies,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships
196553,f:1,FinancialOrg,NaN,Greylock Partners,NaN,operating,1965-01-01,NaN,USA,CA,Menlo Park,SF Bay,2000-03-01,2013-12-05,307,196,NaN,NaN,0,0.0,NaN,NaN,0,71
196554,f:10,FinancialOrg,NaN,Mission Ventures,NaN,operating,1996-01-01,NaN,USA,CA,San Diego,San Diego,2004-10-26,2013-11-22,58,33,NaN,NaN,0,0.0,NaN,NaN,0,11
196555,f:100,FinancialOrg,NaN,"Kapor Enterprises, Inc.",NaN,operating,NaN,NaN,USA,NaN,NaN,TBD,2004-10-28,2006-03-01,2,1,NaN,NaN,0,0.0,NaN,NaN,0,1
196556,f:1000,FinancialOrg,NaN,Speed Ventures,NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,2002-05-27,2002-05-27,1,1
196557,f:10000,FinancialOrg,NaN,3x5 Special Opportunity Partners,NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,2012-10-23,2013-10-29,4,4,NaN,NaN,0,0.0,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208200,f:9995,FinancialOrg,NaN,TxtLoan,NaN,operating,2008-01-01,NaN,GBR,NaN,London,London,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,1
208201,f:9996,FinancialOrg,NaN,founder's Capital,NaN,operating,NaN,NaN,NaN,NaN,NaN,unknown,NaN,NaN,0,0,NaN,NaN,0,0.0,NaN,NaN,0,1
208202,f:9997,FinancialOrg,NaN,Axel Johnson,NaN,operating,1920-01-01,NaN,USA,NY,New York,New York,2012-10-24,2012-10-24,1,1,NaN,NaN,0,0.0,NaN,NaN,0,0
208203,f:9998,FinancialOrg,NaN,Liberty City Ventures,NaN,operating,2012-07-01,NaN,USA,NY,New York,New York,2012-06-01,2013-11-11,5,5,NaN,NaN,0,0.0,NaN,NaN,0,0


In [8]:
funds

,object_id,name,funded_at,raised_amount,raised_currency_code
0,f:371,Second Fund,2008-12-16,300000000.0,USD
1,f:17,Sequoia Israel Fourth Fund,2008-12-17,200750000.0,USD
2,f:951,Tenth fund,2008-08-11,650000000.0,USD
3,f:192,New funds acquire,NaN,625000000.0,USD
4,f:519,Third fund,2008-05-20,200000000.0,USD
...,...,...,...,...,...
1559,f:101,Fund V,2013-12-11,150000000.0,USD
1560,f:7331,JANVEST Technologies LP I,2013-03-19,5.0,USD
1561,f:7791,Fund II,2013-12-11,5000000.0,USD
1562,f:15091,Fund I,2013-12-11,1100000.0,USD


In [9]:
funds_clean = funds[funds['object_id'].isin(objects['object_id'])]
funds_clean

,object_id,name,funded_at,raised_amount,raised_currency_code
0,f:371,Second Fund,2008-12-16,300000000.0,USD
1,f:17,Sequoia Israel Fourth Fund,2008-12-17,200750000.0,USD
2,f:951,Tenth fund,2008-08-11,650000000.0,USD
3,f:192,New funds acquire,NaN,625000000.0,USD
4,f:519,Third fund,2008-05-20,200000000.0,USD
...,...,...,...,...,...
1559,f:101,Fund V,2013-12-11,150000000.0,USD
1560,f:7331,JANVEST Technologies LP I,2013-03-19,5.0,USD
1561,f:7791,Fund II,2013-12-11,5000000.0,USD
1562,f:15091,Fund I,2013-12-11,1100000.0,USD


In [10]:
funds_clean.to_csv(path+'funds_clean.csv')
print('Save Complete')

Save Complete


# Funding Rounds ETL

In [15]:
rounds = pd.read_csv(path +'funding_rounds_clean.csv', index_col=0)
rounds

,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round
0,1,c:4,12/1/2006,series-b,b,8500000,8500000,USD,0,0,NaN,0,0,NaN,2,0,0
1,2,c:5,9/1/2004,angel,angel,500000,500000,USD,0,0,USD,0,0,USD,2,0,1
2,3,c:5,5/1/2005,series-a,a,12700000,12700000,USD,115000000,115000000,USD,0,0,USD,3,0,0
3,4,c:5,4/1/2006,series-b,b,27500000,27500000,USD,525000000,525000000,USD,0,0,USD,4,0,0
4,5,c:7299,5/1/2006,series-b,b,10500000,10500000,USD,0,0,NaN,0,0,NaN,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,57948,c:211890,12/12/2013,series-a,a,3000000,3000000,USD,0,0,USD,0,0,USD,1,1,1
52924,57949,c:267427,2/6/2010,venture,partial,570000,570000,USD,0,0,USD,0,0,USD,0,0,1
52925,57950,c:261728,2/6/2010,venture,unattributed,2184100,2184100,USD,0,0,USD,0,0,USD,0,0,1
52926,57951,c:285864,12/12/2013,series-a,a,790783,790783,USD,0,0,USD,0,0,USD,0,1,1


In [14]:
filled = rounds.fillna(0)
filled

,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round
0,1,c:4,12/1/2006,series-b,b,8500000,8500000,USD,0,0,0,0,0,0,2,0,0
1,2,c:5,9/1/2004,angel,angel,500000,500000,USD,0,0,USD,0,0,USD,2,0,1
2,3,c:5,5/1/2005,series-a,a,12700000,12700000,USD,115000000,115000000,USD,0,0,USD,3,0,0
3,4,c:5,4/1/2006,series-b,b,27500000,27500000,USD,525000000,525000000,USD,0,0,USD,4,0,0
4,5,c:7299,5/1/2006,series-b,b,10500000,10500000,USD,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,57948,c:211890,12/12/2013,series-a,a,3000000,3000000,USD,0,0,USD,0,0,USD,1,1,1
52924,57949,c:267427,2/6/2010,venture,partial,570000,570000,USD,0,0,USD,0,0,USD,0,0,1
52925,57950,c:261728,2/6/2010,venture,unattributed,2184100,2184100,USD,0,0,USD,0,0,USD,0,0,1
52926,57951,c:285864,12/12/2013,series-a,a,790783,790783,USD,0,0,USD,0,0,USD,0,1,1


In [24]:
filled['funded_at'] = filled['funded_at'].replace(0, '1/1/1900')

In [27]:
filled_clean = filled[filled['object_id'].isin(objects['object_id'])]
filled_clean

,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,pre_money_valuation,pre_money_currency_code,post_money_valuation_usd,post_money_valuation,post_money_currency_code,participants,is_first_round,is_last_round
0,1,c:4,12/1/2006,series-b,b,8500000,8500000,USD,0,0,0,0,0,0,2,0,0
1,2,c:5,9/1/2004,angel,angel,500000,500000,USD,0,0,USD,0,0,USD,2,0,1
2,3,c:5,5/1/2005,series-a,a,12700000,12700000,USD,115000000,115000000,USD,0,0,USD,3,0,0
3,4,c:5,4/1/2006,series-b,b,27500000,27500000,USD,525000000,525000000,USD,0,0,USD,4,0,0
4,5,c:7299,5/1/2006,series-b,b,10500000,10500000,USD,0,0,0,0,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52923,57948,c:211890,12/12/2013,series-a,a,3000000,3000000,USD,0,0,USD,0,0,USD,1,1,1
52924,57949,c:267427,2/6/2010,venture,partial,570000,570000,USD,0,0,USD,0,0,USD,0,0,1
52925,57950,c:261728,2/6/2010,venture,unattributed,2184100,2184100,USD,0,0,USD,0,0,USD,0,0,1
52926,57951,c:285864,12/12/2013,series-a,a,790783,790783,USD,0,0,USD,0,0,USD,0,1,1


In [28]:
filled_clean.to_csv(path+'funding_round_filled.csv')
print('Save Complete')

Save Complete
